In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

from FinancialMachineLearning.feature_importance.importance import (
    mean_decrease_impurity,
    mean_decrease_accuracy,
    single_feature_importance,
    plot_feature_importance
)
from FinancialMachineLearning.feature_importance.orthogonal import get_orthogonal_features
from FinancialMachineLearning.machine_learning.cross_validation import PurgedKFold, ml_cross_val_score
from FinancialMachineLearning.multiprocess.multiprocess import process_jobs

/Users/tommylee/Desktop/Pycharm/FinancialMachineLearning/FinancialMachineLearning/multiprocess/fast_ewma.py:5: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((float64[:], int64), nopython = False, nogil = True)


In [2]:
import os
os.chdir('..')
os.chdir('..')
os.getcwd()

'/Users/tommylee/Desktop/Pycharm/FinancialMachineLearning'

In [3]:
matrix = pd.read_csv('./Data/labeled_vertical10.csv', index_col = 0)
matrix

,price,return,fracdiff,volatility,concurrency,uniqueness,side,label
2010-04-29 10:26:57,58.2801,-0.011465,10.476812,0.010964,20.0,0.074445,-1.0,1
2010-04-29 11:09:15,58.3600,-0.012819,10.507715,0.011247,7.0,0.075186,-1.0,1
2010-04-29 11:34:28,58.4100,-0.013664,10.488812,0.011638,5.0,0.067364,-1.0,1
2010-04-29 11:44:01,58.4900,-0.012306,10.540226,0.011979,6.0,0.066014,-1.0,1
2010-04-29 12:00:06,58.4498,-0.014335,10.465450,0.012262,7.0,0.059366,-1.0,1
...,...,...,...,...,...,...,...,...
2023-05-08 15:16:20,152.1563,-0.010951,26.030479,0.010701,46.0,0.020092,-1.0,1
2023-05-08 15:16:37,152.0900,-0.011112,25.963539,0.010595,47.0,0.022152,-1.0,1
2023-05-08 15:36:39,152.0701,-0.010391,25.952489,0.010387,48.0,0.020022,-1.0,1
2023-05-08 15:47:11,152.1800,-0.011105,26.065165,0.010283,49.0,0.020007,-1.0,1


In [4]:
t1 = pd.read_csv('./Data/t1_vertical.csv', index_col = 0)
t1

,t1,trgt,pt,sl
2009-10-02 09:31:33,2009-10-02 10:35:54,0.010011,1,1
2009-10-02 09:44:59,2009-10-02 10:35:54,0.010268,1,1
2009-10-02 09:53:14,2009-10-05 09:59:08,0.010379,1,1
2009-10-02 10:01:56,2009-10-05 09:59:08,0.010430,1,1
2009-10-02 10:09:13,2009-10-05 09:59:08,0.010453,1,1
...,...,...,...,...
2023-05-08 15:16:20,2023-05-10 13:27:11,0.010701,1,1
2023-05-08 15:16:37,2023-05-12 12:21:00,0.010595,1,1
2023-05-08 15:36:39,2023-05-10 13:27:11,0.010387,1,1
2023-05-08 15:47:11,2023-05-10 13:27:11,0.010283,1,1


In [5]:
cont = pd.concat([matrix['label'], t1[-29415:]['trgt'], t1[-29415:]['t1']], axis = 1)
cont.columns = ['bin','w','t1']
cont

,bin,w,t1
2010-04-29 10:26:57,1,0.010964,2010-04-30 15:47:16
2010-04-29 11:09:15,1,0.011247,2010-04-30 15:47:16
2010-04-29 11:34:28,1,0.011638,2010-04-30 15:47:16
2010-04-29 11:44:01,1,0.011979,2010-04-30 14:55:47
2010-04-29 12:00:06,1,0.012262,2010-04-30 15:47:16
...,...,...,...
2023-05-08 15:16:20,1,0.010701,2023-05-10 13:27:11
2023-05-08 15:16:37,1,0.010595,2023-05-12 12:21:00
2023-05-08 15:36:39,1,0.010387,2023-05-10 13:27:11
2023-05-08 15:47:11,1,0.010283,2023-05-10 13:27:11


In [6]:
X_mat = matrix.iloc[:, [0,2,3,4,5]]

In [7]:
from sklearn.svm import SVC

clf = SVC(kernel = 'rbf', probability = True)
param_grid = {'C' : [0.01, 0.1, 1, 10, 100], 'gamma' : [0.01, 0.1, 1, 10, 100]}

In [8]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import time
start = time.time()
gridSearch = GridSearchCV(estimator = clf, param_grid = param_grid, scoring = 'neg_log_loss', n_jobs = -1, cv = 10)
gridSearch.fit(X_mat, cont['bin'], sample_weight = cont['w'])
end = time.time()

print('Calculating time : %.5f second' %(end - start))

Calculating time : 8472.86945 second


In [9]:
from sklearn.model_selection import ParameterGrid
grid = ParameterGrid(param_grid)
num = 0
for i in grid : num += 1

print(num)

25


In [10]:
gridSearch.best_params_

{'C': 0.1, 'gamma': 100}

In [11]:
gridSearch.best_score_

-0.22677724126822446